In [65]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow.keras as keras
import scipy.stats as stats
import math
import os

from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer, QuantileTransformer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from math import sqrt

from tensorflow.keras.layers import TimeDistributed, Attention, Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, BatchNormalization, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from matplotlib.ticker import FuncFormatter

In [66]:
def plot_means_variances(y_true, y_means, y_stddevs):
    plt.rc('font', size=14)
    min_vals = np.min([np.min(y_true), np.min(y_means)])
    max_vals = np.max([np.max(y_true), np.max(y_means)])

    plt.figure(figsize=(16, 6))

    # Plot predicted vs true
    plt.subplot(1, 2, 1)
    plt.scatter(y_true, y_means, alpha = .7, color="0.3", linewidth = 0, s = 2)
    plt.plot([min_vals, max_vals], [min_vals, max_vals], 'k--', color='red')  # Add diagonal line
    plt.title('Fig (a): Predicted vs True Values')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    
    def plot_binned_residuals(y_true, residuals, num_bins=20):
        bins = np.linspace(min(y_true), max(y_true), num_bins + 1)

        bin_means = [0]*num_bins
        bin_stddevs = [0]*num_bins

        for i in range(num_bins):
            mask = (y_true >= bins[i]) & (y_true < bins[i + 1])
            if np.any(mask):
                bin_means[i] = np.mean(y_true[mask])
                bin_stddevs[i] = np.sqrt(mean_squared_error(y_means[mask], y_true[mask]))
        return bin_means, bin_stddevs

    bin_means, bin_stddevs = plot_binned_residuals(y_true, y_means, num_bins=20)
    
    # Plot residuals vs true
    plt.subplot(1, 2, 2)
    plt.scatter(y_true, y_stddevs, alpha = .7, color="0.3", linewidth = 0, s = 2, label='Predicted Standard Deviation', zorder=1)
    plt.scatter(bin_means, bin_stddevs, alpha=1, s=50, color='red', label='True Binned Root Mean Squared Error', zorder=2)
    plt.title('Fig (b): Predicted Standard Deviation vs True RMSE')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Standard Deviation')
    plt.legend()


    plt.tight_layout()
    plt.show()

    
def evaluate_and_print_metrics(results, model_name, y_train, y_test, y_train_pred, y_test_pred, y_train_stddevs, y_test_stddevs, ci):
    z_value = stats.norm.ppf((1 + ci) / 2)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    train_mae = mean_absolute_error(y_train, y_train_pred)    # in %
    test_mae = mean_absolute_error(y_test, y_test_pred)       # in %

    train_lower_bound = y_train_pred - z_value * y_train_stddevs
    train_upper_bound = y_train_pred + z_value * y_train_stddevs

    test_lower_bound = y_test_pred - z_value * y_test_stddevs
    test_upper_bound = y_test_pred + z_value * y_test_stddevs

    train_within_interval = np.sum(np.logical_and(y_train.ravel() >= train_lower_bound, y_train.ravel() <= train_upper_bound))
    test_within_interval = np.sum(np.logical_and(y_test.ravel() >= test_lower_bound, y_test.ravel() <= test_upper_bound))

    train_percentage_within_interval = (train_within_interval / len(y_train.ravel())) * 100
    test_percentage_within_interval = (test_within_interval / len(y_test.ravel())) * 100

   
    results[model_name] = {
        "Test Root Mean Squared Error (RMSE): ": test_rmse,
        "Test Mean Absolute Error (MAE): ": test_mae,
        f"Percentage of Test Data Points within {ci*100:.2f}% CI: ": test_percentage_within_interval
    }

    print(f"Train RMSE: {train_rmse:.3f}")
    print(f"Test RMSE: {test_rmse:.3f}")
    print(f"Train MAE: {train_mae:.3f}")
    print(f"Test MAE: {test_mae:.3f}")
    print(f"Percentage of Train Data Points within {ci*100:.2f}% CI: {train_percentage_within_interval:.2f}%")
    print(f"Percentage of Test Data Points within {ci*100:.2f}% CI: {test_percentage_within_interval:.2f}%")
    
def plot_confidence_interval_scatter(y_test_pred, y_test_std, y_test, bins=20):
    plt.rc('font', size=14)
    
    # Compute the t-values of the confidence intervals based on Z-scores
    t_values = np.array([stats.norm.ppf(i/bins + (1-i/bins)/2) for i in range(1, bins+1)])

    percentages_within_interval = []
    for t_value in t_values:
        lower_bounds = y_test_pred.ravel() - t_value * y_test_std
        upper_bounds = y_test_pred.ravel() + t_value * y_test_std

        # Count number of data points within the confidence interval
        is_within_interval = np.logical_and(y_test >= lower_bounds, y_test <= upper_bounds)
        num_within_interval = np.sum(is_within_interval)

        # Calculate the percentage of data points within the confidence interval
        percentage_within_interval = (num_within_interval / len(y_test)) * 100
        percentages_within_interval.append(percentage_within_interval)

    plt.figure(figsize=(8, 8))
    plt.scatter(np.arange(1, bins+1)*100/bins, percentages_within_interval, color='blue', label='Percentage of Residuals within Interval')
    
    # Plot the expected diagonal line (red line)
    plt.plot([0, 100], [0, 100], color='red', linestyle='--', label='Expected')

    # Add percentage symbols to x-axis ticks
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x)}%'))

    plt.xlabel('Confidence Intervals')
    plt.ylabel('Percentage within Interval')
    plt.title('Scatter Plot of Percentage of Residuals within the Confidence Intervals')
    plt.legend()

    plt.show()
    
def load_dataset_train_test_split(df, features, output_feature):
    X = df[features]
    y = df[output_feature]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Scale input data to facilitate training
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, np.array(y_train), np.array(y_test), scaler

def plot_loss_history(history):
    plt.plot(history.history['loss'][1:], label='Training Loss')
    plt.plot(history.history['val_loss'][1:], label='Validation Loss', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
    
def compute_predictions(model, X_train, X_test, num_samples=100):
    y_train_pred = []
    y_test_pred = []
    for _ in range(num_samples):
        y_train_pred.append(model.predict(X_train))
        y_test_pred.append(model.predict(X_test))
        
    y_train_pred = np.concatenate(y_train_pred, axis=1)
    y_test_pred = np.concatenate(y_test_pred, axis=1)

    y_train_pred_mean = np.mean(y_train_pred, axis=1)
    y_train_pred_stddevs = np.std(y_train_pred, axis=1)
    
    y_test_pred_mean = np.mean(y_test_pred, axis=1)
    y_test_pred_stddevs = np.std(y_test_pred, axis=1)
    
    return y_train_pred_mean, y_train_pred_stddevs, y_test_pred_mean, y_test_pred_stddevs

def NLL(y, distr): 
    return -distr.log_prob(y) 

# We add 0.001 to the standard deviation to ensure it does not converge to 0 and destabilizes training because the gradient
# of maximum likelihood estimation requires the inversion of the variance. We also activate the parameters using a softplus
# activation function to enfore a positive standard deviation estimate.
def normal_softplus(params): 
    return tfd.Normal(loc=params[:, 0:1], scale=1e-3 + tf.math.softplus(0.05 * params[:, 1:2]))

In [67]:
# In order to ensure that each model has repeatable results,we fix the seed both for the
# data splitting part and for the initilialization of the networks' weights. Theoretially
# speaking, we should average over different seeds to ensure the robustness of our results.
# However, in practice, due to the size of the data set this is unfeasibile and we only do
# this for the best performing model to show that the variability of results based on seed
# is almost none.

keras.utils.set_random_seed(812)
MODELS_SEED = 42

In [68]:
file_path = 'Cleaned_data.pkl'
df_full = pd.read_pickle(file_path)

In [69]:
# Datetime column
DATETIME_COL = 'Date.time'

# Features considered
features = [
'Wind.speed.me',
'Wind.speed.sd',
'Wind.speed.min',
'Wind.speed.max',
'Front.bearing.temp.me',
'Front.bearing.temp.sd',
'Front.bearing.temp.min',
'Front.bearing.temp.max',
'Rear.bearing.temp.me',
'Rear.bearing.temp.sd',
'Rear.bearing.temp.min',
'Rear.bearing.temp.max',
'Rotor.bearing.temp.me',
'Stator1.temp.me',
'Nacelle.ambient.temp.me',
'Nacelle.temp.me',
'Transformer.temp.me',
'Gear.oil.temp.me',
'Gear.oil.inlet.temp.me',
'Top.box.temp.me',
'Hub.temp.me',
'Conv.Amb.temp.me',
'Rotor.bearing.temp.me',
'Transformer.cell.temp.me',
'Motor.axis1.temp.me',
'Motor.axis2.temp.me',
'CPU.temp.me',
'Blade.ang.pitch.pos.A.me',
'Blade.ang.pitch.pos.B.me',
'Blade.ang.pitch.pos.C.me',
'Gear.oil.inlet.press.me',
'Gear.oil.pump.press.me',
'Drive.train.acceleration.me',
'Tower.Acceleration.x',
'Tower.Acceleration.y'
]

output_feature = 'Power.me'
TURBINE_ID = 5

df = df_full
print(f"Total data points before removing NaNs: ", len(df))
df = df.dropna(subset=features + [output_feature] + [DATETIME_COL])
print(f"Total data points after removing NaNs: ", len(df))
df = df.reset_index(drop=False)

def train_test_split_by_turbine(group, test_size=0.2):
    train_set, test_set = train_test_split(group, test_size=test_size, random_state=42)
    return train_set, test_set

splits = df.groupby('turbine').apply(train_test_split_by_turbine)

df_pretrain_train = pd.concat([split[0] for split in splits.tolist()])
df_pretrain_test = pd.concat([split[1] for split in splits.tolist()])
print("Pre-train Training Set Size: ", df_pretrain_train.shape[0])
print("Pre-train Testing Set Size: ", df_pretrain_test.shape[0])

TURBINE_ID = 5
df_finetune_train = splits[TURBINE_ID][0]
df_finetune_test = splits[TURBINE_ID][1]
print("Finetune Training Set Size: ", df_finetune_train.shape[0])
print("Finetune Testing Set Size: ", df_finetune_test.shape[0])

Total data points before removing NaNs:  1018494
Total data points after removing NaNs:  1009707
Pre-train Training Set Size:  807764
Pre-train Testing Set Size:  201943
Finetune Training Set Size:  151504
Finetune Testing Set Size:  37877


In [70]:
def create_design_matrix(df_train, df_test, features, output_feature):
    X_train, y_train = df_train[features].to_numpy(), df_train[output_feature].to_numpy()
    X_test, y_test = df_test[features].to_numpy(), df_test[output_feature].to_numpy()

    # Scale input data to facilitate training
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler
    
X_train_full, X_test_full,\
    y_train_full, y_test_full,\
    scaler_full = create_design_matrix(df_pretrain_train, df_pretrain_test, features, output_feature)

X_train_single_turbine, X_test_single_turbine, \
    y_train_single_turbine, y_test_single_turbine, \
    scaler_single_turbine = create_design_matrix(df_finetune_train, df_finetune_test, features, output_feature)

In [71]:
# Define the file path for saving the weights
checkpoint_path = 'saved_models/pretrain.h5'

# Define the initial model architecture
def generic_model(X_train_full):
    inputs = Input(shape=(X_train_full.shape[1],))
    hidden1 = Dense(300, activation="relu")(inputs)
    hidden2 = Dense(200, activation="relu")(hidden1)
    hidden3 = Dense(100, activation="relu")(hidden2)

    params = Dense(2)(hidden3)

    dist = tfp.layers.DistributionLambda(normal_softplus)(params)

    model = Model(inputs=inputs, outputs=dist)
    model.compile(Adam(learning_rate=0.001), loss=NLL)

    return model

# Train the initial model using X_full with the checkpoint callback
generic_model = generic_model(X_train_full)
generic_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 35)]              0         
                                                                 
 dense_21 (Dense)            (None, 300)               10800     
                                                                 
 dense_22 (Dense)            (None, 200)               60200     
                                                                 
 dense_23 (Dense)            (None, 100)               20100     
                                                                 
 dense_24 (Dense)            (None, 2)                 202       
                                                                 
 distribution_lambda_4 (Dis  ((None, 1),               0         
 tributionLambda)             (None, 1))                         
                                                           

In [ ]:
# Define the callback to save the weights
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,
                                      monitor='val_loss', mode='min', save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# Load weights from the checkpoint if available
if os.path.exists(checkpoint_path):
    print("Checkpoint found. Loading weights.")
    generic_model.load_weights(checkpoint_path)
    # Get the number of epochs already run
    start_epoch = generic_model.history.epoch[-1]
    print(f"Resuming training from epoch {start_epoch}.")
else:
    print("No checkpoint found. Training from scratch.")
    start_epoch = 0

history = generic_model.fit(X_train_full, y_train_full, epochs=500, batch_size=64, initial_epoch=start_epoch,
                            validation_split=0.1, callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/500
11360/11360 [==============================] - 54s 5ms/step - loss: 882.2598 - val_loss: 4.7424
Epoch 2/500
11360/11360 [==============================] - 45s 4ms/step - loss: 4.6531 - val_loss: 4.6093
Epoch 3/500
11360/11360 [==============================] - 42s 4ms/step - loss: 4.4392 - val_loss: 4.4240
Epoch 4/500
11360/11360 [==============================] - 42s 4ms/step - loss: 4.3106 - val_loss: 4.3377
Epoch 5/500
11360/11360 [==============================] - 43s 4ms/step - loss: 4.2118 - val_loss: 4.1565
Epoch 6/500
11360/11360 [==============================] - 47s 4ms/step - loss: 4.1414 - val_loss: 4.0575
Epoch 7/500
11360/11360 [==============================] - 35s 3ms/step - loss: 4.0890 - val_loss: 4.0645
Epoch 8/500
11360/11360 [==============================] - 39s 3ms/step - loss: 4.0467 - val_loss: 4.0506
Epoch 9/500
11360/11360 [==============================] - 48s 4ms/step - loss: 4.0133 - val_loss: 4.0926
Epoch 10/500
11360/11360 [==================

In [ ]:
generic_model.load_weights(checkpoint_path)
evaluation = generic_model.evaluate(X_test_full, y_test_full)
print("Evaluation Loss:", evaluation)

In [ ]:
def create_model_finetune(X_train, generic_model, seed):
    keras.utils.set_random_seed(seed)

    inputs = Input(shape=(X_train.shape[1],))

    # Step 1: Load the architecture and weights of the previously trained model
    pretrained_model_layers = generic_model.layers[1:]
    l = inputs

    for layer in pretrained_model_layers:
        layer.trainable = True
        l = layer(l)

    model_mlp_gaussian = Model(inputs=inputs, outputs=l)
    model_mlp_gaussian.compile(Adam(learning_rate=1e-4), loss=NLL)

    return model_mlp_gaussian

model_finetune = create_model_finetune(X_train_single_turbine, generic_model, MODELS_SEED)
model_finetune.summary()

In [ ]:
# Define the callback to save the weights
checkpoint_callback = ModelCheckpoint(filepath='saved_models/finetuned.keras', save_weights_only=True,
                                      monitor='val_loss', mode='min', save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model_finetune.fit(X_train_single_turbine, y_train_single_turbine, epochs=50, batch_size=32,
                            validation_split=0.1, callbacks=[checkpoint_callback, early_stopping_callback])
plot_loss_history(history)

In [ ]:
# generic_model.load_weights(checkpoint_path)
evaluation = model_finetune.evaluate(X_test_single_turbine, y_test_single_turbine)
print("Evaluation Loss:", evaluation)

In [ ]:
y_train_pred = np.array(model_finetune(X_train_single_turbine).mean()).ravel()
y_test_pred = np.array(model_finetune(X_test_single_turbine).mean()).ravel()

y_train_stddevs = np.array(model_finetune(X_train_single_turbine).stddev()).ravel()
y_test_stddevs = np.array(model_finetune(X_test_single_turbine).stddev()).ravel()

evaluate_and_print_metrics({}, f"Fine Tuned",
y_train_single_turbine, y_test_single_turbine, y_train_pred, y_test_pred,
y_train_stddevs, y_test_stddevs, 0.99)

In [ ]:
plot_means_variances(y_test_single_turbine, y_test_pred, y_test_stddevs)

In [ ]:
plot_confidence_interval_scatter(y_test_pred, y_test_stddevs, y_test_single_turbine, bins=20)